In [1]:
%config IPCompleter.greedy=True
!pip install azureml-pipeline-wrapper --extra-index-url https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/12903033 --upgrade 
# then you need to restart kernel

In [1]:
from azureml.core import Workspace, Run, Dataset
from azureml.pipeline.wrapper import Pipeline, dsl, Module

ws = Workspace.get(name='kubeflow_ws_1', subscription_id='74eccef0-4b8d-4f83-b5f9-fa100d155b22', resource_group='kubeflow-demo')

In [5]:
# Load modules from workspace
#train_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Train')
#score_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Score')
#eval_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Evaluate')

# Load a module from local as anonymous module
#a_module_in_development_func = Module.load_from_yaml(ws, path='/home/code/new_module/spec.yaml')
train_module_func = Module.from_yaml(ws, './train-score-eval/train.yaml')
score_module_func = Module.from_yaml(ws, './train-score-eval/score.yaml')
eval_module_func = Module.from_yaml(ws, './train-score-eval/eval.yaml')

train_data = Dataset.get_by_name(ws, 'training_data')
test_data = Dataset.get_by_name(ws, 'test_data')

In [6]:
help(train_module_func)

Help on function [module] Train (namespace: microsoft.com/aml/samples) in [module] Train (namespace: microsoft:

[module] Train (namespace: microsoft.com/aml/samples)(training_data: 'Training data organized in the torchvision format/structure' = None, maxrundurationseconds: 'MaxRunDurationSeconds' = '', max_epochs: 'Maximum number of epochs for the training' = None, learning_rate: 'Learning rate, default is 0.01' = '0.01')
    :param training_data: Training data organized in the torchvision format/structure
    :type training_data: ['AnyFile', 'AnyDirectory']
    :param maxrundurationseconds: MaxRunDurationSeconds
    :type maxrundurationseconds: str
    :param max_epochs: Maximum number of epochs for the training
    :type max_epochs: int
    :param learning_rate: Learning rate, default is 0.01. (optional, default value is 0.01.)
    :type learning_rate: float
    
    
    :output model_output: model_output
    :type: model_output: AnyFile



In [8]:
@dsl.pipeline(
    name='A simple pipeline',
    description='A sample',
#    default_compute_target='kubeflow-aks'
    default_compute_target='cpu-cluster'
)
def sample_pipeline():
    train = train_module_func(
        training_data=train_data, 
        max_epochs=5, 
        learning_rate=0.02)

    score = score_module_func(
        model_input=train.outputs.model_output, 
        test_data=test_data)

    eval = eval_module_func(scoring_result=score.outputs.score_output)

    return {**eval.outputs}

p = sample_pipeline()
run = p.submit(experiment_name='jietong')

run.wait_for_completion()

Submitted PipelineRun 3559357c-9164-46e6-947b-e582a3ee382a
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/jietong/runs/3559357c-9164-46e6-947b-e582a3ee382a?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/kubeflow-demo/workspaces/kubeflow_ws_1
PipelineRunId: 3559357c-9164-46e6-947b-e582a3ee382a
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/jietong/runs/3559357c-9164-46e6-947b-e582a3ee382a?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/kubeflow-demo/workspaces/kubeflow_ws_1
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 9a0ec55e-1e89-48e1-a5e0-e495c5cb95f9
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/jietong/runs/9a0ec55e-1e89-48e1-a5e0-e495c5cb95f9?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/kubeflow-demo/workspaces/kubeflow_ws_1
StepRun( Train ) Status: NotStarted
StepRun( Train ) Status: Queued
StepRun( Trai

<RunStatus.completed: 'Completed'>